<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Biblio_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tokenitzadors**

---
Si no hi ha un model d'idioma disponible en l'idioma que ens interessa, o si el nostre corpus és molt diferent del que s'ha entrenat el vostre model d'idioma, probablement necessitarem entrenar el model des de zero mitjançant un tokenitzador adaptat a les nostres dades. Això requerirà entrenar un nou tokenitzador pel conjunt de dades.


Es treballa amb un tokenizer personalitzat o amb un model, sovint es fa el següent:

1. Es carrega un dataset (pot ser de text, bibliografia, etc.).

1. Es construeix un corpus a partir d’aquest dataset.

1. Es fa servir aquest corpus per:

  * entrenar un tokenizer,

  * o tokenitzar el text per passar-lo a un model.

In [ ]:
!pip install datasets


from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

In [ ]:
raw_datasets["train"]

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

# **Llavors… per què generar un corpus si ja tenim un dataset?**

---


El dataset pot contenir:

1. Metadades,

1. Camps diversos (autor, títol, any, resum…),

1. Informació textual estructurada o no estructurada.

Però el corpus generalment és:

Una llista de textos (normalment només cadenes de caràcters) que serveixen com a entrada per entrenar o aplicar un tokenizer.

In [4]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [ ]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

In [6]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

# **Entrenant un nou tokenitzador**

---
Ara que tenim el nostre corpus en forma d'iterador de lots de textos, estem preparats per entrenar un nou tokenitzador. Per fer-ho, primer hem de carregar el tokenitzador que volem emparellar amb el nostre model (aquí, GPT-2). Tot i que entrenarem un nou tokenitzador, és una bona idea fer-ho per evitar començar completament des de zero. D'aquesta manera, no haurem d'especificar res sobre l'algorisme de tokenització o els fitxes especials que volem utilitzar; el nou tokenitzador serà exactament el mateix que GPT-2, i l'únic que canviarà és el vocabulari, que vindrà determinat per la formació del nostre corpus.


In [20]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

El tokenitzador té uns quants símbols especials, com Ġi Ċ, que denoten espais i noves línies, respectivament. S'observa que, això no és massa eficient: el tokenitzador retorna fitxes individuals per a cada espai, quan podria agrupar nivells de sagnat (ja que tenir conjunts de quatre o vuit espais serà molt habitual al codi). També va dividir el nom de la funció de manera una mica estranya, sense estar acostumat a veure paraules amb el _personatge.

Anem a formar un nou tokenizer i veure si soluciona aquests problemes

In [22]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

La funció **train_new_from_iterator** no forma part de la classe base AutoTokenizer directament, sinó que:

Està disponible en tokenitzadors basats en la llibreria tokenizers (com GPT2TokenizerFast), i

Només funciona si el tokenizer carregat és del tipus "Fast tokenizer", és a dir, una instància de PreTrainedTokenizerFast

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

In [ ]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

In [ ]:
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")